In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np

In [18]:
#data = pd.read_csv('pmdata.csv')

data = pd.read_csv('PRSA_data_2010.1.1-2014.12.31_1.csv', header=0)


In [19]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [20]:

data = data.dropna()
# data = pd.get_dummies(data)

In [21]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [24]:
wind_direction_mapping = {
    'NW': 1,
    'NE': 2,
    'SE': 3,
    'cv': 4
}

# Replace the string categories with the numeric values
data['cbwd'] = data['cbwd'].replace(wind_direction_mapping)

C:\Users\lyxzz\AppData\Local\Temp\ipykernel_16248\3534025218.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cbwd'] = data['cbwd'].replace(wind_direction_mapping)


In [26]:
train_data_by_month, test_data_by_month = {}, {}
train_data_by_year, test_data_by_year = {}, {}

# Splitting data for each month (month-year combination) separately
for year in data['year'].unique():
    for month in data['month'].unique():
        month_year_data = data[(data['year'] == year) & (data['month'] == month)]
        train_data_by_month[(year, month)], test_data_by_month[(year, month)] = train_test_split(month_year_data,
                                                                                                 test_size=0.2, random_state=12345)

# Combine the monthly splits into yearly splits
for year in data['year'].unique():
    yearly_train_data = pd.concat([train_data_by_month[(year, month)] for month in data['month'].unique()])
    yearly_test_data = pd.concat([test_data_by_month[(year, month)] for month in data['month'].unique()])
    train_data_by_year[year] = yearly_train_data
    test_data_by_year[year] = yearly_test_data

# Combine all monthly splits into a single dataset for full range modeling
train_data_full = pd.concat(train_data_by_month.values())
test_data_full = pd.concat(test_data_by_month.values())

In [27]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,3,1.79,0,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,3,2.68,0,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,3,3.57,0,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,3,5.36,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,3,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,1,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,1,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,1,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [28]:
test_data_by_year.items()

dict_items([(2010,         No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES  cbwd     Iws  \
264    265  2010      1   12     0   21.0   -22 -15.0  1031.0     4    1.78   
487    488  2010      1   21     7   18.0   -25  -9.0  1038.0     1  138.10   
528    529  2010      1   23     0   49.0   -17  -4.0  1033.0     1  172.08   
438    439  2010      1   19     6  330.0    -4  -1.0  1021.0     1   12.07   
685    686  2010      1   29    13   12.0   -24   5.0  1022.0     1   50.07   
...    ...   ...    ...  ...   ...    ...   ...   ...     ...   ...     ...   
8023  8024  2010     12    1     7  245.0    -1   0.0  1015.0     2    0.89   
8720  8721  2010     12   30     8   10.0   -24  -8.0  1025.0     1  190.44   
8470  8471  2010     12   19    22  126.0   -14   6.0  1018.0     3    6.26   
8757  8758  2010     12   31    21   19.0   -21  -6.0  1031.0     1  552.53   
8407  8408  2010     12   17     7   40.0   -17  -6.0  1014.0     1    0.89   

      Is  Ir  
264    0   0  
48

In [33]:
year_training_results = {}
year_testing_results = {}
year_prediction_vectors = pd.DataFrame()
year_model_bw = {}
col_drop = ['pm2.5', 'No', 'year']


for year, train_data in train_data_by_year.items():
    test_data = test_data_by_year[year]
    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # model detail
    nw_model = sm.nonparametric.KernelReg(endog=y_train, exog=X_train, var_type='c' * X_train.shape[1],
                                      bw='cv_ls',ckertype='gaussian',reg_type='lc')

    
    # Make predictions and calculate metrics
    year_model_bw[year] = nw_model.bw
    train_pred,mar_train = nw_model.fit(X_train)
    test_pred,mar_test = nw_model.fit(X_test)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    # Store and print results
    year_training_results[year] = train_rmse
    year_testing_results[year] = test_rmse
    train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
    year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
    print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)


Year: 2010, Training RMSE: 16.38361495465862, Testing RMSE: 24.058247778763604


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x


Year: 2011, Training RMSE: 6.553151895835373, Testing RMSE: 20.18299936354514
Year: 2012, Training RMSE: 13.538962595078402, Testing RMSE: 23.211669875075494
Year: 2013, Training RMSE: 5.751020040979228, Testing RMSE: 20.794904309417134


D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:238: RuntimeWarning: invalid value encountered in divide
  G = G_numer / G_denom
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:250: RuntimeWarning: invalid value encountered in divide
  B_x = d_mx / f_x - G * d_fx / f_x
D:\Anaconda_Python\Anaconda3\lib\site-packages\statsmodels\nonparametric\kernel_regression.py:251: RuntimeWarning: invalid value encountered in divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)


Year: 2014, Training RMSE: 8.458104519117624, Testing RMSE: 20.998300381290356


In [34]:
year_model_bw

{2010: array([ 0.1874792 , -0.15059313,  4.7356413 ,  2.70807877,  2.28724853,
         1.9215153 ,  1.98891983,  8.08573991,  1.58658511,  1.91437245]),
 2011: array([-0.08466622, -0.09141404,  0.85491031,  4.22834838,  3.45271226,
         2.42201955,  1.93285157, 23.50543799,  1.05100343,  1.55137442]),
 2012: array([ 0.05071787,  0.18042877,  3.13937037,  1.87341462, 11.8899097 ,
         6.33492307,  1.11800427,  4.34283811,  0.38362789,  2.02405246]),
 2013: array([-0.16080252, -0.09435443,  0.71102616,  7.08634217,  6.60753877,
         3.04352066,  1.334434  , 52.88534833,  0.78040562,  0.25067791]),
 2014: array([-0.25145255, -0.42031444,  2.97809602, -2.83999158,  1.85549106,
         0.72955121,  1.91652979,  7.30999471,  1.02195759,  1.71740159])}

In [45]:
year_training_results

{2010: 16.38361495465862,
 2011: 6.553151895835373,
 2012: 13.538962595078402,
 2013: 5.751020040979228,
 2014: 8.458104519117624}

In [48]:
year_testing_results

{2010: 24.058247778763604,
 2011: 20.18299936354514,
 2012: 23.211669875075494,
 2013: 20.794904309417134,
 2014: 20.998300381290356}

In [46]:
year_training_results_df =  pd.DataFrame([year_training_results])
year_training_results_df.to_csv('year_training_results.csv', index=False)

In [47]:
year_testing_results_df =  pd.DataFrame([year_testing_results])
year_testing_results_df.to_csv('year_testing_results.csv', index=False)

In [42]:
year_prediction_vectors
year_prediction_vectors.to_csv('year_prediction_vectors.csv', index=False)

In [41]:
year_model_bw_df  = pd.DataFrame(year_model_bw)
year_model_bw_df.to_csv('year_model_bw.csv', index=False)